In [65]:
import pandas as pd
import torch
import os
import json
import pymorphy2
import re
import nltk
import cv2
import numpy as np
import shutil

from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer
from nltk.translate import meteor
from nltk import word_tokenize, sent_tokenize
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from nltk.translate.nist_score import sentence_nist
from torchvision import models, transforms
from torchvision.models.resnet import ResNet50_Weights
from PIL import Image

In [66]:
def del_timestamps(text):
    text = text.split("] ")[1:]
    return " ".join(text)

def ret_stt(stt_name):
    with open(f"./train_stt/{stt_name}", 'r', encoding="utf_8_sig") as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
    return lines

In [67]:
def process_corpus_(data, corpus_index):
    data["stt"] = data["stt_name"].apply(ret_stt)

    txt = data["stt"][corpus_index]
    
    tmp = []
    flag = 0
    flag0 = 0
    
    for i in txt:
        
        for j in i:
            if j == '*':
                flag = 1
            else:
                continue
        if flag == 0 and len(i.split()) > 5:
            tmp.append(i)
        else:
            flag = 0

    
    return pd.Series(tmp)

In [68]:
def process_corpus(file_name):
    data = pd.read_csv(file_name)
    data = data.head(20)
    data["stt_sum"] = [process_corpus_(data, i) for i in range(data.shape[0])]
    return data

In [69]:
class CustomDataset(Dataset):

    def __init__(self, X):
        self.text = X

    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=150)

    def __len__(self):
        return self.text.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        return {k: v.reshape(-1) for k, v in output.items()}


In [70]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output['last_hidden_state']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [71]:
tokenizer = BertTokenizer.from_pretrained('content/rubert_cased_L-12_H-768_A-12_pt')
model = BertModel.from_pretrained('content/rubert_cased_L-12_H-768_A-12_pt', output_hidden_states=True)
device = torch.device("cpu")
model.to(device)
model.eval()
tokenizer1 = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')
model1 = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
device='cpu'
model1.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(50364, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
       

In [72]:
def calc(corpus):
    eval_ds = CustomDataset(corpus)
    eval_dataloader = DataLoader(eval_ds, batch_size=10)
    if len(corpus) > 50:

        embeddings = torch.Tensor().to(device)

        with torch.no_grad():
            for n_batch, batch in enumerate(tqdm(eval_dataloader)):
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(**batch)
                embeddings = torch.cat([embeddings, mean_pooling(outputs, batch['attention_mask'])])
            embeddings = embeddings.cpu().numpy()

        pca = PCA(n_components=15, random_state=42)
        emb_15d = pca.fit_transform(embeddings)

        kmeans = KMeans(n_clusters=10, random_state=42)
        cluster_labels = kmeans.fit_predict(emb_15d)
        cluster_centers = kmeans.cluster_centers_
        unique_clusters = np.unique(cluster_labels)

        cluster_centers_indices = {}
        for cluster_label in unique_clusters:
            cluster_centers_indices[cluster_label] = np.where(cluster_labels == cluster_label)[0][0]

        tmp = []
        for i in cluster_centers_indices.values():
            tmp.append(i)
        tmp.sort()

        line1 = ""
        core_sentences = []
        for i in range(len(tmp)//2):
            line1 = line1 + corpus[tmp[i]] + f" <extra_id_{i}>"
            core_sentences.append(corpus[tmp[i]])

        line2 = ""
        for i in range(len(tmp)//2, len(tmp)):
            line2 = line2 + corpus[tmp[i]] + f" <extra_id_{i - len(tmp)//2}>"
            core_sentences.append(corpus[tmp[i]])

        lm_text="Заполни пробелы: " + line1
        input_ids=torch.tensor([tokenizer1.encode(lm_text)]).to(device)
        outputs=model1.generate(input_ids,eos_token_id=tokenizer1.eos_token_id,early_stopping=True)
        replace_dict = {
            match.group(): replacement
            for match, replacement in zip(re.finditer(r'<extra_id_\d+>', tokenizer1.decode(outputs[0][1:])),
                                          re.split(r'<extra_id_\d+>', tokenizer1.decode(outputs[0][1:]))[1:])
        }

        def replacer(match):
            return replace_dict.get(match.group(), '')

        result1 = re.sub(r'<extra_id_\d+>', replacer, line1)

        lm_text="Заполни пробелы: " + line2
        input_ids=torch.tensor([tokenizer1.encode(lm_text)]).to(device)
        outputs=model1.generate(input_ids,eos_token_id=tokenizer1.eos_token_id,early_stopping=True)
        replace_dict = {
        match.group(): replacement
            for match, replacement in zip(re.finditer(r'<extra_id_\d+>', tokenizer1.decode(outputs[0][1:])),
                                          re.split(r'<extra_id_\d+>', tokenizer1.decode(outputs[0][1:]))[1:])
        }

        def replacer(match):
            return replace_dict.get(match.group(), '')

        result2 = re.sub(r'<extra_id_\d+>', replacer, line2)

        final_line = result1 + result2

        return final_line
    
    else:
        return ""

In [73]:
def show_examples(cluster, n):
    print(corpus[emb_2d['label'] == cluster][::])

In [74]:
def meteor_metric(text, text_sum):
    if isinstance(text_sum, str):
        return round(meteor([word_tokenize(text)],word_tokenize(text_sum)), 4)
    else:
        return 0

In [75]:
def bleu_metric(reference, hypothesis):
    reference = [word_tokenize(reference)]
    hypothesis = word_tokenize(hypothesis)
    return round(sentence_bleu(reference, hypothesis), 4)

In [76]:
def nist_metric(reference, hypothesis):
    try:
        reference = [word_tokenize(reference)]
        hypothesis = word_tokenize(hypothesis)
        return round(sentence_nist(reference, hypothesis), 4)
    except ZeroDivisionError:
        return 0  # или другое значение по умолчанию

In [77]:
data = process_corpus("train.csv")

C:\Users\druzh\AppData\Local\Temp\ipykernel_4700\2763201655.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series(tmp)


In [78]:
data["desc_proc"] = [calc(i) for i in data["stt_sum"].values]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:50<00:00,  1.81s/it]
C:\Users\druzh\Project_python\venv\lib\site-packages\transformers\generation\configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
C:\Users\druzh\Project_python\venv\lib\site-packages\transformers\generation\utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:17<00:00,  1.58s/it]
C:\Users\druzh\Project_python\venv\lib\site-packages\transformers\generation\configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
C:\Users\druzh\Project_python\venv\lib\site-packages\transformers\generation\utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [79]:
data.head()

,video_name,stt_name,category_name,title,description,stt,stt_sum,desc_proc
0,0.mp4,0.txt,Развлечения,Правильная цена I #3,С вами Макс Климток и это шоу Правильная цена!...,"[ Давай поспорим на выпуск, вот типа, что я то...","0 Давай поспорим на выпуск, вот типа, чт...","Давай поспорим на выпуск, вот типа, что я точ..."
1,1.mp4,1.txt,Спорт/Игры,Три лошадиные силы | Выпуск №2,В этом новом выпуске нас ждут не менее новые и...,"[ ты поедешь со мной в тверь, это 2 выпуск, ...",0 ты поедешь со мн...,ты поедешь со мной в тверь ? Заполни пробелы...
2,2.mp4,2.txt,Блоги,Хашлама | Выпуск 4 | Силиконовый ПРЕСС Давы | ...,"Привет, это Султан и Авет! Мы опять хаваем вку...","[ Добрый вечер, дорогие друзья, с вами шоу «Хо...","0 Добрый вечер, дорогие друзья, с вами ш...","Добрый вечер, дорогие друзья, с вами шоу «Хош..."
3,3.mp4,3.txt,Путешествия,Прогулка по стране - Владивосток,Прогулка по Владивостоку. Самому большому горо...,"[ СПОКОЙНАЯ МУЗЫКА, Редактор субтитров Е.Воин...","Series([], dtype: float64)",
4,4.mp4,4.txt,Искусство,Артмеханика. Выпуск 3. Татуировки + Mika Vino,Были ли татуировки на теле Николая II? Почему ...,"[ ДИНАМИЧНАЯ МУЗЫКА, ДИНАМИЧНАЯ МУЗЫКА, АПЛО...","0 Здравствуйте, дорогие друзья! Мы по ва...","Здравствуйте, дорогие друзья! Мы по вам, по в..."


In [80]:
data["met"] = data.apply(lambda x: meteor_metric(x.description, x.desc_proc), axis=1)

In [81]:
data.met.mean()

0.11294500000000003

In [82]:
data["bleu"] = data.apply(lambda x: bleu_metric(x.description, x.desc_proc), axis=1)

C:\Users\druzh\Project_python\venv\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\druzh\Project_python\venv\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\druzh\Project_python\venv\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or us

In [83]:
data["nist"] = data.apply(lambda x: nist_metric(x.description, x.desc_proc), axis=1)

In [84]:
data["nist"].values

array([0.4623, 0.3914, 0.6305, 0.    , 0.6475, 0.0615, 0.4222, 0.8587,
       0.345 , 1.0224, 0.    , 0.    , 0.5519, 0.5458, 0.8412, 0.5266,
       0.532 , 0.5068, 1.3808, 0.4985])

In [86]:
data.nist.mean()

0.511255